<a href="https://colab.research.google.com/github/victoria-1989/Projet1_wild_code_school/blob/main/Projet_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

KPI Ventes

In [ ]:
WITH prec_mois_table AS(
SELECT P.productLine,
YEAR(O.orderDate) AS Annee,
MONTH(O.orderDate) AS Mois,
DATE_FORMAT(O.orderDate, '%Y-%m') AS Periode,
sum(quantityOrdered) AS nombre_produits_vendus
FROM orders O
LEFT JOIN orderdetails OD ON O.orderNumber = OD.orderNumber
LEFT JOIN products P ON OD.productCode=P.productCode WHERE status="shipped" OR status="resolved"
GROUP BY productLine,Periode,Annee,Mois ORDER BY Periode,productLine,nombre_produits_vendus DESC)

SELECT p1.productLine,p1.Annee,p1.Mois,p1.Periode,p1.nombre_produits_vendus,
p2.Annee,p2.Mois,p2.Periode,p2.nombre_produits_vendus
FROM prec_mois_table AS p1
LEFT JOIN prec_mois_table AS p2 ON p1.productLine=p2.productLine AND p1.Annee=p2.Annee+1 AND p1.Mois=p2.Mois

KPI Finances

Nota bene : parfois les indicateurs métiers ne sont pas techniquement réalisables. C'est à vous de l'expliquer, et d'apporter vos propres idées pour répondre aux besoins de l'entreprise.

Le chiffre d'affaires des commandes des deux derniers mois par pays:

In [ ]:
# Nous n'avons pas toutes les données pour répondre à cette question. Mais nous pouvons le réaliser sur une année.
SELECT MONTH (o.orderDate) AS month,
YEAR (o.orderDate) AS year,
country,
SUM(p.amount) AS CA
FROM payments p
JOIN customers c ON c.customerNumber = p.customerNumber
JOIN orders o ON o.customerNumber = c.customerNumber
GROUP BY country, month, year
ORDER BY year, month

In [ ]:
# KPI CA
WITH ca_global AS(
SELECT
YEAR (p.paymentDate) AS year,
MONTH (p.paymentDate) AS month,
country,
SUM(p.amount) AS CA
FROM payments p
JOIN customers c ON c.customerNumber = p.customerNumber
GROUP BY country, year, month
ORDER BY country, year)

, taux_global AS(
SELECT *,
LAG(CA) OVER ( partition by country ORDER BY year) AS CA_global_moinN
FROM ca_global)

SELECT *,
ROUND((((CA - CA_global_moinN)/CA_global_moinN)),4) AS evolution
FROM taux_global

#KPI CA evolution
WITH ca_global AS(
SELECT
YEAR (p.paymentDate) AS year,
country,
SUM(p.amount) AS CA
FROM payments p
JOIN customers c ON c.customerNumber = p.customerNumber
GROUP BY country, year
ORDER BY country, year)

, taux_global AS(
SELECT *,
LAG(CA) OVER ( partition by country ORDER BY year) AS CA_global_moinN
FROM ca_global)

SELECT *,
ROUND((((CA - CA_global_moinN)/CA_global_moinN)),2) AS evolution
FROM taux_global

In [ ]:
"""Problématique et conseil
Il faudrait rattacher la table paiment à la table order, pour avoir un meilleur tracking des données et donc mieux centraliser et utiliser les données les données.

Commandes qui n'ont pas encore été payées - On peut sortir le montant par client on ne peut pas faire ressortir les numéros de commande

In [ ]:
WITH table_temp AS (

SELECT sum(quantityOrdered*priceEach) AS Montant_commande,o.customerNumber,c.customerName AS nomclient,c.salesRepEmployeeNumber AS referent
FROM orderdetails od
JOIN orders o ON od.orderNumber = o.orderNumber
JOIN customers c ON o.customerNumber = c.customerNumber
GROUP BY customerNumber),

payment_customer AS (

SELECT customerNumber, sum(amount) AS Commande_payee
FROM payments
GROUP BY customerNumber)


SELECT table_temp.customerNumber,
table_temp.nomclient,
table_temp.referent,
table_temp.Montant_commande,
payment_customer.Commande_payee,
table_temp.Montant_commande - payment_customer.Commande_payee AS Difference
FROM table_temp
JOIN payment_customer ON table_temp.customerNumber = payment_customer.customerNumber
WHERE table_temp.Montant_commande<>payment_customer.Commande_payee

KPI Logistiques

Recherche des 5 produits les plus commandés et une jointure pour retrouver les références par productCode

In [ ]:
SELECT p.productCode, p.productName, p.quantityInStock, SUM(od.quantityOrdered) AS totalOrdered
FROM products p
JOIN orderdetails od ON p.productCode = od.productCode
GROUP BY p.productCode, p.productName, p.quantityInStock
ORDER BY totalOrdered DESC
LIMIT 5;


Recherche des 5 produits les moins commandés et une jointure pour retrouver les références par productCode

In [ ]:
SELECT p.productCode, p.productName, p.quantityInStock, SUM(od.quantityOrdered) AS totalOrdered
FROM products p
JOIN orderdetails od ON p.productCode = od.productCode
GROUP BY p.productCode, p.productName, p.quantityInStock
ORDER BY totalOrdered ASC
LIMIT 5;


KPI Ressources Humaines

In [ ]:
#KPI RH
WITH top_employee AS (
SELECT MONTH (p.paymentDate) AS month,
YEAR (p.paymentDate) AS year,
CONCAT (e.firstname, " ", e.lastname) AS name,
ROUND (SUM(p.amount),0) AS CA,
e.officeCode, employeeNumber, reportsTo
FROM payments p
JOIN customers c ON p.customerNumber = c.customerNumber
JOIN employees e ON c.salesRepEmployeeNumber = e.employeeNumber
GROUP BY name,month, year, officeCode, employeeNumber, reportsTo
ORDER BY CA DESC)

SELECT *,
city
from top_employee
join offices on top_employee.officeCode = offices.officeCode

#KPI RH Name manager
with name_report AS (
SELECT e.reportsTO,
CONCAT (e2.firstname, " ", e2.lastname) AS name_manager
FROM employees e
LEFT JOIN employees e2 ON e2.`employeeNumber` = e.`reportsTo`)

SELECT *
FROM name_report